In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statistics
###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
###############################################
from bokeh.plotting import figure , show
from pybaselines import whittaker as pl

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
file_path = 'CR1000Y/CR1000Y-12.csv'

Sample_Spectra = pd.read_csv(file_path)
Sample_Spectra = Sample_Spectra.rename({' intensity':'Intensities'},axis=1)


Sample_Spectra_Plot = figure(title = 'Raw Sample Spectra' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Plot.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="red" )
Sample_Spectra_Plot.width = 1200
Sample_Spectra_Plot.height = 600

print (Sample_Spectra)
show (Sample_Spectra_Plot)

       wavelength  Intensities
0      180.000000          0.0
1      180.033333          0.0
2      180.066667          0.0
3      180.100000          0.0
4      180.133333          0.0
...           ...          ...
23426  960.866667          0.0
23427  960.900000          0.0
23428  960.933333          0.0
23429  960.966667          0.0
23430  961.000000          0.0

[23431 rows x 2 columns]


In [4]:
#https://pybaselines.readthedocs.io/en/latest/api/pybaselines/index.html

base,_ = pl.drpls(Sample_Spectra.Intensities)

Sample_Spectra_Baseline = figure(title = 'Corrected Baseline in hLIBS Spectra', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Baseline.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="red" )
# Sample_Spectra_Baseline.line(Sample_Spectra.wavelength,base , line_width = 2, color ="red" )
Sample_Spectra_Baseline.line(Sample_Spectra.wavelength,(Sample_Spectra.Intensities - base) , line_width = 2, color ="green" )

Corrected_Baseline_Spectra = pd.DataFrame()
Corrected_Baseline_Spectra['wavelength']= Sample_Spectra.wavelength
Corrected_Baseline_Spectra['Intensities'] = (Sample_Spectra.Intensities-base)
print(Corrected_Baseline_Spectra)

Sample_Spectra_Baseline.width = 1200
Sample_Spectra_Baseline.height = 600
show(Sample_Spectra_Baseline)
print(Sample_Spectra_Baseline)



       wavelength   Intensities
0      180.000000 -2.387968e-04
1      180.033333 -2.244969e-04
2      180.066667 -2.101920e-04
3      180.100000 -1.958724e-04
4      180.133333 -1.815244e-04
...           ...           ...
23426  960.866667  1.075459e-06
23427  960.900000  9.678466e-07
23428  960.933333  8.601545e-07
23429  960.966667  7.524259e-07
23430  961.000000  6.446866e-07

[23431 rows x 2 columns]


figure(id='p1045', ...)


In [5]:
Normalization_Intensity = max(Corrected_Baseline_Spectra.Intensities)
print(Normalization_Intensity)

27560.615391625095


In [6]:
Normalized_Spectra = pd.DataFrame()
Normalized_Spectra['wavelength']= Corrected_Baseline_Spectra.wavelength
Normalized_Spectra['Intensities'] = (Corrected_Baseline_Spectra.Intensities / Normalization_Intensity)
print(Normalized_Spectra)

       wavelength   Intensities
0      180.000000 -8.664422e-09
1      180.033333 -8.145570e-09
2      180.066667 -7.626533e-09
3      180.100000 -7.106967e-09
4      180.133333 -6.586371e-09
...           ...           ...
23426  960.866667  3.902160e-11
23427  960.900000  3.511702e-11
23428  960.933333  3.120956e-11
23429  960.966667  2.730077e-11
23430  961.000000  2.339159e-11

[23431 rows x 2 columns]


In [7]:
Selected_data = Normalized_Spectra[(Normalized_Spectra['wavelength'] >= 238.0) & (Normalized_Spectra['wavelength'] <= 238.55)]
# print(Selected_data)
height = None
Peak_Prominenace = None #(RMS_intensity_noise + Threshold)  #Uses S/N ratio , as comapred to other peaks
minimum_distance = None   #minimum distance between two peaks

Selected_data_peak , _ = fp(Selected_data.Intensities,height=height,prominence=Peak_Prominenace,distance=minimum_distance)
Selected_Sample_Spectra = Selected_data.iloc[Selected_data_peak] 
print(Selected_Sample_Spectra)


Selected_data_Plot = figure(title = 'Selected Data Spectra' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Selected_data_Plot.line(Selected_data.wavelength,Selected_data.Intensities , line_width = 2, color ="red" )
Selected_data_Plot.circle(Selected_Sample_Spectra.wavelength, Selected_Sample_Spectra.Intensities , color = 'green' , size = 8)

Selected_data_Plot.width = 400
Selected_data_Plot.height = 400
show (Selected_data_Plot)

      wavelength  Intensities
1746       238.2      0.89284
